In [1]:
import os
import numpy as np
import pandas as pd
import torch

from torch_geometric.data import Data

## Load dataset

The Wiki-talk dataset we will use is not available in the ```torch_geometric``` package. Thus we need to load it manually.

In [2]:
# params
data_dir = 'data/wiki-talk/'
train_size = 0.5
lang = 'sk'    # specify which language of Wikipedia 

### Load edge info

In [3]:
edgelist = pd.read_csv(os.path.join(data_dir, f"{lang}-wiki-talk"), sep='\t', header=None, names=["source", "target", "timestamp"])
edgelist

,source,target,timestamp
0,8,8,2003-12-03T09:04:10Z
1,14,14,2004-02-23T09:10:57Z
2,16,14,2004-02-23T09:28:02Z
3,16,14,2004-02-25T20:41:16Z
4,28,14,2004-04-05T16:58:09Z
...,...,...,...
131879,66699,121620,2015-11-21T21:16:33Z
131880,66699,121883,2015-11-22T15:25:22Z
131881,39673,121883,2015-11-22T16:25:03Z
131882,12161,121883,2015-11-22T18:57:16Z


### Load node info

In [4]:
df_nodes = pd.read_csv(os.path.join(data_dir, f"{lang}-user-group"), sep='\t', header=None)
df_nodes

,0,1
0,14940,1
1,36018,1
2,9422,0
3,74846,1
4,47824,1
...,...,...
148,12656,1
149,16434,1
150,4036,1
151,27162,2


In [5]:
max_node_num = np.max(edgelist[["source", "target"]].values) + 1
node_ids = np.sort(np.unique(edgelist[["source", "target"]].values))

In [6]:
# number of nodes
N = len(node_ids)

In [7]:
# dataset (roles)
roles = df_nodes[[0, 1]].values

y = [0] * max_node_num
for r in roles:
    y[r[0]] = r[1]

y = np.array([y[i] for i in node_ids])

In [8]:
df_nodes = pd.DataFrame({'id': node_ids, 'label': y})
df_nodes

,id,label
0,2,0
1,8,0
2,9,0
3,12,0
4,14,0
...,...,...
41447,121707,0
41448,121750,0
41449,121851,0
41450,121883,0


## Convert to continuous node IDs
In the original Wiki-talk datasets, the node IDs are discontinuous as shown above, because they represent the original Wikipedia user IDs. However, graph neural networks typically expect node indices to range from 0 to `N-1` where `N` is the number of nodes.
Thus, the conversion is necessary. We now show a way to convert the discontinous node IDs to continuous ones.

In [9]:
# create a mapping between the original IDs and continuous IDs
node_id_mapping = { node_id : i for i, node_id in enumerate(node_ids) }

In [10]:
# replace the original IDs with the continuous IDs
edgelist[["source", "target"]] = edgelist[["source", "target"]].replace(node_id_mapping)
df_nodes['id'] = df_nodes['id'].replace(node_id_mapping)

## Compute node features

In [11]:
import networkx as nx

# we can use directed graph here because we do not rely on the node classification algorithms from nx
G = nx.DiGraph()
G.add_nodes_from(range(N))

In [12]:
G.add_edges_from(edgelist[["source", "target"]].values)

In [13]:
# Compute Degree
deg = G.degree()

# Compute In-Degree
in_deg = G.in_degree()

# Compute Out-Degree
out_deg = G.out_degree()

# Compute Clustering Coefficient
clust_coeff = nx.clustering(G.to_undirected())

# Compute PageRank
pr = nx.pagerank(G, alpha=0.85)

In [14]:
len(edgelist[edgelist['source'] == 28]['target'].unique())

159

In [17]:
out_deg[28]

159

In [16]:
G.degree

DiDegreeView({0: 6, 1: 2, 2: 3, 3: 10, 4: 26, 5: 2, 6: 3, 7: 2, 8: 2, 9: 2, 10: 2, 11: 15, 12: 2, 13: 44, 14: 2, 15: 26, 16: 2, 17: 2, 18: 2, 19: 7, 20: 2, 21: 1, 22: 1, 23: 1, 24: 215, 25: 3, 26: 3, 27: 2, 28: 453, 29: 8, 30: 2, 31: 13, 32: 29, 33: 2, 34: 5, 35: 133, 36: 2, 37: 5, 38: 2, 39: 2, 40: 2, 41: 1, 42: 2, 43: 2, 44: 2, 45: 37, 46: 2, 47: 2, 48: 3, 49: 15, 50: 5, 51: 2, 52: 21, 53: 16, 54: 6, 55: 7, 56: 4, 57: 4, 58: 2, 59: 2, 60: 2, 61: 1, 62: 2, 63: 5, 64: 2, 65: 4, 66: 2, 67: 1, 68: 6, 69: 4, 70: 2, 71: 236, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 3, 78: 95, 79: 89, 80: 614, 81: 2, 82: 3, 83: 2, 84: 3, 85: 2, 86: 2, 87: 2, 88: 13, 89: 3, 90: 38, 91: 873, 92: 1, 93: 2, 94: 2, 95: 1, 96: 1, 97: 1, 98: 2, 99: 4, 100: 21, 101: 1, 102: 20, 103: 3, 104: 2, 105: 7, 106: 1, 107: 4, 108: 2, 109: 8, 110: 2, 111: 3, 112: 999, 113: 23, 114: 618, 115: 3, 116: 3, 117: 125, 118: 2, 119: 96, 120: 2, 121: 1, 122: 12, 123: 2, 124: 3, 125: 2, 126: 1, 127: 3, 128: 1, 129: 4, 130: 7, 131: 83, 1

### Split training and test data

In [11]:
# Generate random permutation of node indices
perm = torch.randperm(N)

# Select train and test nodes
train_idx = perm[: int(train_size * N)]
test_idx = perm[int(train_size * N) :]

# Initialize train_mask and test_mask with False
train_mask = torch.zeros(N, dtype=torch.bool)
test_mask = torch.zeros(N, dtype=torch.bool)

# Set the selected indices to True
train_mask[train_idx] = True
test_mask[test_idx] = True

### Load data to PyG

In [12]:
edge_index = torch.tensor(edgelist[["target", "source"]].values.T, dtype=torch.long)

# node features and labels
x = torch.tensor(np.ones([N, 1]), dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

In [13]:
data = Data(x=x, y=y, edge_index=edge_index, train_mask=train_mask, test_mask=test_mask)
data

Data(x=[41452, 1], edge_index=[2, 131884], y=[41452], train_mask=[41452], test_mask=[41452])

In [14]:
data.validate(raise_on_error=True)

True

## Define and train a simply Graph Convolutional Network model

In [15]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [16]:
# find the best device to run on
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# move the model and data to the device
model = GCN().to(device)
data = data.to(device)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

## Evaluation

To evaluate the Graph Convolutional Network model we just trained, we use it to predict the labels of the nodes in the test set, and compare the results with the ground truth. We print the accuracy as the metrics.

In [19]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.9972


In [20]:
torch.sum(pred[data.test_mask])

tensor(0, device='mps:0')